In [1]:
import mlflow
import pandas as pd
from deployments import get_chat_model, AvailableChatModels
from evaluation import create_llamacpp_eval_function


/mnt/c/Users/ms101/Desktop/dev/DnDLLM/env/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/mnt/c/Users/ms101/Desktop/dev/DnDLLM/env/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/mnt/c/Users/ms101/Desktop/dev/DnDLLM/env/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
eval_data = pd.DataFrame(
    {
        "inputs": [
            "What is MLflow?",
            "What is Spark?",
        ],
        "ground_truth": [
            "MLflow is an open-source platform for managing the end-to-end machine learning (ML) "
            "lifecycle. It was developed by Databricks, a company that specializes in big data and "
            "machine learning solutions. MLflow is designed to address the challenges that data "
            "scientists and machine learning engineers face when developing, training, and deploying "
            "machine learning models.",
            "Apache Spark is an open-source, distributed computing system designed for big data "
            "processing and analytics. It was developed in response to limitations of the Hadoop "
            "MapReduce computing model, offering improvements in speed and ease of use. Spark "
            "provides libraries for various tasks such as data ingestion, processing, and analysis "
            "through its components like Spark SQL for structured data, Spark Streaming for "
            "real-time data processing, and MLlib for machine learning tasks",
        ],
    }
)

In [3]:
llm = get_chat_model(model_name=AvailableChatModels.LLAMA_32_3B_Q8, max_new_tokens=500)

/mnt/c/Users/ms101/Desktop/dev/DnDLLM/src/llm/.model_cache/llama-3.2-3b-instruct-q8_0.gguf


llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /mnt/c/Users/ms101/Desktop/dev/DnDLLM/src/llm/.model_cache/llama-3.2-3b-instruct-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_model_loader: - kv   6:                               general.tags arr[str,6]       = ["

In [4]:
llamacpp_qa = create_llamacpp_eval_function(llm)

In [5]:
# Evaluate the custom LLM using mlflow.evaluate
with mlflow.start_run():
    results = mlflow.evaluate(
        model=llamacpp_qa,
        data=eval_data,
        targets="ground_truth",  # Column in eval_data containing ground truth answers
        model_type="question-answering",  # Specify task type
    )


    # Evaluation result for each data record is available in `results.tables`.
    eval_table = results.tables["eval_results_table"]
    print(f"See evaluation table below: \n{eval_table}")

2024/11/16 14:57:25 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.

llama_print_timings:        load time =     872.56 ms
llama_print_timings:      sample time =      16.88 ms /   374 runs   (    0.05 ms per token, 22151.15 tokens per second)
llama_print_timings: prompt eval time =     872.50 ms /    12 tokens (   72.71 ms per token,    13.75 tokens per second)
llama_print_timings:        eval time =    6565.90 ms /   373 runs   (   17.60 ms per token,    56.81 tokens per second)
llama_print_timings:       total time =    7742.11 ms /   385 tokens
Llama.generate: 9 prefix-match hit, remaining 2 prompt tokens to eval

llama_print_timings:        load time =     872.56 ms
llama_print_timings:      sample time =       9.48 ms /   211 runs   (    0.04 ms per token, 22257.38 tokens per second)
llama_print_timings: prompt eval time =      19.62 ms /     2 tokens (    9.81 ms per token,   101.95 tokens per second)
llama_print_timings:        eval time =    3553.

MlflowException: Metric 'exact_match': Error:
ValueError("Classification metrics can't handle a mix of binary and unknown targets")
Traceback (most recent call last):
  File "/mnt/c/Users/ms101/Desktop/dev/DnDLLM/env/lib/python3.12/site-packages/mlflow/models/evaluation/default_evaluator.py", line 1710, in _test_first_row
    metric_value = _evaluate_metric(metric_tuple, eval_fn_args)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/ms101/Desktop/dev/DnDLLM/env/lib/python3.12/site-packages/mlflow/models/evaluation/default_evaluator.py", line 670, in _evaluate_metric
    metric = metric_tuple.function(*eval_fn_args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/ms101/Desktop/dev/DnDLLM/env/lib/python3.12/site-packages/mlflow/metrics/metric_definitions.py", line 162, in _accuracy_eval_fn
    acc = accuracy_score(y_true=targets, y_pred=predictions, sample_weight=sample_weight)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^